In [1]:
import nltk
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nicolas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nicolas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nicolas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Nicolas\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import datetime

In [3]:
from string import punctuation
from gensim import models
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
import re
import spacy
import enchant


In [4]:
train=pd.read_csv("train.csv",encoding='latin-1')
datos =pd.read_csv("test.csv",encoding='latin-1')
datos.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
#Separo el target de los tweets
target = train.iloc[:,-1]

In [6]:
#Aplico operaciones sobre train que luego replicare en el set de datos, para obtener los indices que yo considero relevantes
#y deshacerme de los que no lo son
#Empiezo por las arroba
train['cantidad@']=train['text'].str.count('@')
datos['cantidad@']=datos['text'].str.count('@')
train.head()

,id,keyword,location,text,target,cantidad@
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0


In [7]:
#Aplico una funcion de hash sobre la location
#Primero fijo los NaN como la cadena "ninguna"
#Luego proceso eliminando mayusculas haciendo lemmatize
def reemplazarTexto(x):
  x=x.casefold()
  return x

def procesarLocation(df):
  df['location']=df['location'].fillna(value='none')
  return df['location'].apply(lambda x:reemplazarTexto(x))
train['location']=procesarLocation(train)
datos['location']=procesarLocation(datos)

nlp = spacy.load("en_core_web_sm")
def proc(texts):
  l=''
  if (texts=='none'):
    return 'none'
  tokens=nltk.word_tokenize(texts)
  for doc in nlp.pipe(tokens, disable=["tagger"]):
      for ent in doc.ents:
        if ([(ent.text,ent.label_)]!=[]) & (ent.label_=='GPE'):
            l+= ent.text+','
  if l=='':
    return 'no valida'
  return l
train['locationLimpia']=train['location'].apply(lambda x:proc(x))
datos['locationLimpia']=datos['location'].apply(lambda x:proc(x))

In [8]:
train['ubicacionValida']= ((train['locationLimpia']!='no valida') & (train['locationLimpia']!='none'))*1
datos['ubicacionValida']= ((datos['locationLimpia']!='no valida') & (datos['locationLimpia']!='none'))*1

In [9]:
def tokenizarTexto(x):
  x=x.casefold()
  d = enchant.Dict("en")
  x=re.sub(r'http\S*', '',x)
  x=re.sub(r'[^a-z\s]', '',x)
  lemma=WordNetLemmatizer()
  x=lemma.lemmatize(x)
  tokens=nltk.word_tokenize(x)
  #tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
  #tokens = tknzr.tokenize(x)
  l=stopwords.words('english')+list(punctuation)
  tokens = [word for word in tokens if word not in l]
  tokens= [word for word in tokens if d.check(word)]
  tokens= [word for word in tokens if wordnet.synsets(word)]
  return tokens

In [10]:
#Analizo longitud de los tweets
train['longitudTweet']=train.text.str.len()
datos['longitudTweet']=datos.text.str.len()

In [11]:
#Analizo la cantidad de Hashtags en un tweet
train['cantidad#']=train['text'].str.count('#')
datos['cantidad#']=datos['text'].str.count('#')

In [12]:
#Cuento si el texto contiene un http y cuantos tiene
def analizarHttp(x):
  tokens=nltk.word_tokenize(x)
  resultado=0
  for token in tokens:
    resultado += ((token=='http') | (token=='https'))*1      
  return resultado

train['cantidadLinks']=train['text'].apply(lambda x:analizarHttp(x))
datos['cantidadLinks']=datos['text'].apply(lambda x:analizarHttp(x))

In [13]:
#Proceso cada hashtag
def procesarHashtag(x):
  tokens=nltk.word_tokenize(x)
  listaHashtags=[]
  for i in range(len(tokens)):
    if ((tokens[i]=='#') & (i!=len(tokens)-1)):
      listaHashtags.append(tokens[i+1])
  return listaHashtags

train['hashtagsVector']=train['text'].apply(lambda x:procesarHashtag(x))
datos['hashtagsVector']=datos['text'].apply(lambda x:procesarHashtag(x))

In [14]:
#Analizo si los hashtags contienen alguna keyword
def analizarHashtag(x,keys):
  tokens=nltk.word_tokenize(x)
  lemma=WordNetLemmatizer()
  resultado=False
  for i in range(len(tokens)):
    if ((tokens[i]=='#') & (i!=len(tokens)-1)):
      resultado = lemma.lemmatize(tokens[i+1]) in keys
  return resultado*1
lTrainK=train['keyword'].unique().tolist()
lDatosK=train['keyword'].unique().tolist()
train['hashtagKey'] = train['text'].apply(lambda x:analizarHashtag(x,lTrainK))
datos['hashtagKey'] = datos['text'].apply(lambda x:analizarHashtag(x,lDatosK))

In [15]:
#Analizo si se menciona algun tipo de entretenimiento en el tweet
#Puede ser juegos, musica, peliculas...
listaEntretenimiento=['music','radio','tv','show','dj','game','gaming','videogame','history','movies','youtube','book','ebook','sports','esports']

def analizarContenido(x):
  tokens=nltk.word_tokenize(x)
  resultado=0
  for palabra in tokens:
    if palabra in listaEntretenimiento:
      resultado = 1
  return resultado

train['entretenimiento'] = train['text'].apply(lambda x:analizarContenido(x))
datos['entretenimiento'] = datos['text'].apply(lambda x:analizarContenido(x))

In [16]:
def orgHash(texts):
  loc=0
  for doc in nlp.pipe(texts, disable=["tagger"]):
      for ent in doc.ents:
        if (ent.label_=='ORG'):
          loc=1
  return loc
train['locationEnHashtag']=train['hashtagsVector'].apply(lambda x:orgHash(x))
datos['locationEnHashtag']=datos['hashtagsVector'].apply(lambda x:orgHash(x))

In [17]:
#Mean encoding por keyword
mean=train['target'].mean()
agg=train.groupby('keyword')['target'].agg(['count','mean'])
counts=agg['count']
means=agg['mean']
weight=100
smooth=(counts*means+weight*mean)/(counts+weight)
train['SmoothKey']=train['keyword'].map(smooth)
datos['SmoothKey']=datos['keyword'].map(smooth)

In [18]:
datos['SmoothKey'].fillna(value=datos['SmoothKey'].mean(),inplace=True)
train['SmoothKey'].fillna(value=train['SmoothKey'].mean(),inplace=True)

In [19]:
#Mean encoding por location
mean=train['target'].mean()
agg=train.groupby('locationLimpia')['target'].agg(['count','mean'])
counts=agg['count']
means=agg['mean']
weight=100
smooth=(counts*means+weight*mean)/(counts+weight)
train['SmoothLocation']=train['locationLimpia'].map(smooth)
datos['SmoothLocation']=datos['locationLimpia'].map(smooth)

In [20]:
datos['SmoothLocation'].fillna(value=datos['SmoothLocation'].mean(),inplace=True)
train['SmoothLocation'].fillna(value=train['SmoothLocation'].mean(),inplace=True)

In [21]:
#Freq encoding
train['hashtag'] = train['hashtagsVector'].apply(lambda x: ' '.join(x))
datos['hashtag'] = datos['hashtagsVector'].apply(lambda x: ' '.join(x))
fe=train.groupby('hashtag').size()/len(train)
train['hashtagFreqEnc']=train['hashtag'].map(fe)
datos['hashtagFreqEnc']=datos['hashtag'].map(fe)
train['hashtagFreqEnc'].fillna(value=train['hashtagFreqEnc'].median(),inplace=True)
datos['hashtagFreqEnc'].fillna(value=datos['hashtagFreqEnc'].median(),inplace=True)

In [22]:
#Analizo la relacion entre cantidad de # y la longitud del tweet
train['relacion#longitud']=train['cantidad#']/train['longitudTweet']*100
datos['relacion#longitud']=datos['cantidad#']/datos['longitudTweet']*100

In [23]:
#Analizo la relacion entre cantidad de @ y la longitud del tweet
train['relacion@longitud']=train['cantidad@']/train['longitudTweet']*100
datos['relacion@longitud']=datos['cantidad@']/datos['longitudTweet']*100

In [24]:
train['relacionTodoLongitud']=(train['cantidad@']+train['cantidad#']+train['cantidadLinks'])/train['longitudTweet']*100
datos['relacionTodoLongitud']=(datos['cantidad@']+datos['cantidad#']+datos['cantidadLinks'])/datos['longitudTweet']*100

In [25]:
#Analizo la relacion entre cantidad de @ y la cantidad de #
train['suma@y#']=train['cantidad@']+train['cantidad#']
datos['suma@y#']=datos['cantidad@']+datos['cantidad#']

In [26]:
#Analizo cantidad de palabras
train['cantidadPalabras']=train['text'].apply(lambda x: len(tokenizarTexto(x)))
datos['cantidadPalabras']=datos['text'].apply(lambda x: len(tokenizarTexto(x)))

In [27]:
#Mean encoding por catidad de palabras
mean=train['target'].mean()
agg=train.groupby('cantidadPalabras')['target'].agg(['count','mean'])
counts=agg['count']
means=agg['mean']
weight=100
smooth=(counts*means+weight*mean)/(counts+weight)
train['SmoothCantPal']=train['cantidadPalabras'].map(smooth)
datos['SmoothCantPal']=datos['cantidadPalabras'].map(smooth)

In [28]:
train['SmoothCantPal'].fillna(value=train['SmoothCantPal'].median(),inplace=True)
datos['SmoothCantPal'].fillna(value=datos['SmoothCantPal'].median(),inplace=True)

In [29]:
#Mean encoding por catidad de @ y #
mean=train['target'].mean()
agg=train.groupby('suma@y#')['target'].agg(['count','mean'])
counts=agg['count']
means=agg['mean']
weight=100
smooth=(counts*means+weight*mean)/(counts+weight)
train['Smooth@y#']=train['suma@y#'].map(smooth)
datos['Smooth@y#']=datos['suma@y#'].map(smooth)

In [30]:
datos['Smooth@y#'].fillna(value=datos['Smooth@y#'].median(),inplace=True)
train['Smooth@y#'].fillna(value=train['Smooth@y#'].median(),inplace=True)

In [31]:
import math
train['longitudTweet']=train['longitudTweet'].apply(lambda x: round(x/10))
datos['longitudTweet']=datos['longitudTweet'].apply(lambda x: round(x/10))

In [32]:
#Mean por long
mean=train['target'].mean()
agg=train.groupby('longitudTweet')['target'].agg(['count','mean'])
counts=agg['count']
means=agg['mean']
weight=120
smooth=(counts*means+weight*mean)/(counts+weight)
train['SmoothLongitud']=train['longitudTweet'].map(smooth)
datos['SmoothLongitud']=datos['longitudTweet'].map(smooth)

In [33]:
datos['SmoothLongitud'].fillna(value=datos['SmoothLongitud'].mean(),inplace=True)
train['SmoothLongitud'].fillna(value=train['SmoothLongitud'].mean(),inplace=True)

In [34]:
#Remuevo cosas que no hacen al analisis
entrenar=train.drop(columns=['id','keyword','location','text','hashtagsVector','locationLimpia','hashtag','suma@y#','cantidad#','cantidad@','cantidadLinks'])
test=datos.drop(columns=['id','keyword','location','text','hashtagsVector','locationLimpia','hashtag','suma@y#','cantidad#','cantidad@','cantidadLinks'])

In [35]:
entrenar.to_csv('entrenar.csv',index=False)
test.to_csv('testear.csv',index=False)